In [150]:
# Importing libarys
import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

How is your prediction task defined? And what is the meaning of the output variable?

In [136]:
# Reading credit dataset
data = pd.read_csv('credit_dataset.csv', index_col=0)

In [137]:
# Perform one-hot encoding
dummies = pd.get_dummies(data[["Gender", "Education", "Marriage Status"]])

# Concatenate the original data with the encoded dummies
data_enc = pd.concat([data, dummies], axis=1)

# Do feature engineering to get the total amount for bill amount and pay amount; then find the amount each customer owes the bank
data_enc["total bill"] = data_enc["Bill_Amount1"] + data_enc["Bill_Amount2"] + data_enc["Bill_Amount3"]
data_enc["total paid"] = data_enc["Pay_Amount1"] + data_enc["Pay_Amount2"] + data_enc["Pay_Amount3"]
data_enc["outstanding payable"] = data_enc["total bill"] - data_enc["total paid"]

# Create a subset to be dropped
to_drop = ["Gender", "Education", "Marriage Status","Bill_Amount1", "Bill_Amount2", "Bill_Amount3", "Pay_Amount1", "Pay_Amount2", "Pay_Amount3","Gender_male"]

#Drop the data for feature selection
data_enc = data_enc.drop(to_drop, axis = 1)

In [138]:

data["overall_amount1"] = data["Bill_Amount1"] - data["Pay_Amount1"]
data["overall_amount2"] = data["Bill_Amount2"] - data["Pay_Amount2"]
data["overall_amount3"] = data["Bill_Amount3"] - data["Pay_Amount3"]
amounts = data[["overall_amount1","overall_amount2","overall_amount3"]]

# Iterate over each column in the DataFrame
for column in amounts.columns:
    # Check if the column contains numeric values
    if pd.api.types.is_numeric_dtype(amounts[column]):
        # Update positive values to 1 and negative values to 0
        amounts[column] = amounts[column].apply(lambda x: 1 if x > 0 else 0)

# Calculate the sum of 1s and 0s for each row
data_enc["unpaid_months"] = amounts.sum(axis=1)

# Print the row sums
print(data_enc.head())



             Credit Limit  Age  Default payment next month  Gender_female  \
Customer ID                                                                 
1                   20000   24                           1              1   
2                  120000   26                           1              1   
3                   90000   34                           0              1   
4                   50000   37                           0              1   
5                   50000   57                           0              0   

             Education_high school  Education_post-graduate  \
Customer ID                                                   
1                                0                        0   
2                                0                        0   
3                                0                        0   
4                                0                        0   
5                                0                        0   

             Educa

C:\Users\3mm3t\AppData\Local\Temp\ipykernel_15476\3571068290.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amounts[column] = amounts[column].apply(lambda x: 1 if x > 0 else 0)


In [ ]:
# Create a StandardScaler object
scaler = MinMaxScaler()

# Select the numerical columns to be scaled
numerical_columns = ['total bill','total paid','outstanding payable']

# Scale the numerical columns
data_enc[numerical_columns] = scaler.fit_transform(data_enc[numerical_columns])

In [141]:
# plt.hist(data_enc["total bill"], bins = 100)
# plt.boxplot(data_enc["total bill"])
# plt.show()

In [142]:
# from sklearn.preprocessing import MinMaxScaler

# # Create a StandardScaler object
# scaler = MinMaxScaler()

# # Select the numerical columns to be scaled
# numerical_columns = ['total bill','total paid','outstanding payable']

# # Scale the numerical columns
# data_enc[numerical_columns] = scaler.fit_transform(data_enc[numerical_columns])

# # Print the scaled data
# plt.hist(data_enc["total bill"], bins = 100)
# plt.boxplot(data_enc["total bill"])
# plt.show()


In [143]:
# import numpy as np

# # Apply logarithmic transformation to "total_bill"
# data_enc['total_bill_log'] = np.log1p(data_enc['total bill'])

# # Print the transformed data


In [144]:
# plt.boxplot(data_enc["total_bill_log"])
# plt.show()

In [145]:
# Split the data into features (X) and target variable (y)
X = data_enc.drop("Default payment next month", axis=1)
y = data_enc["Default payment next month"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
random_forest = RandomForestClassifier()

# Train the model
random_forest.fit(X_train, y_train)

# Make predictions on the test set
y_pred = random_forest.predict(X_test)

# Perform cross-validation
scores = cross_val_score(random_forest, X, y, cv=5)

# Print the cross-validation scores
print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", np.mean(scores))


Cross-Validation Scores: [0.784375 0.75625  0.7625   0.765625 0.778125]
Mean Accuracy: 0.7693749999999999


In [146]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Split the data into features (X) and target variable (y)
X = data_enc.drop("Default payment next month", axis=1)
y = data_enc["Default payment next month"]

# Initialize the SGDClassifier
sgd_classifier = SGDClassifier()

# Perform cross-validation
scores = cross_val_score(sgd_classifier, X, y, cv=5)

# Print the cross-validation scores
print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", np.mean(scores))

Accuracy: 0.753125
Cross-Validation Scores: [0.7875   0.7875   0.721875 0.7875   0.7875  ]
Mean Accuracy: 0.774375


In [147]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Split the data into features (X) and target variable (y)
X = data_enc.drop("Default payment next month", axis=1)
y = data_enc["Default payment next month"]

# Initialize the SGDClassifier
KNN = KNeighborsClassifier()

# Perform cross-validation
scores = cross_val_score(KNN, X, y, cv=5)

# Print the cross-validation scores
print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", np.mean(scores))

Accuracy: 0.753125
Cross-Validation Scores: [0.740625 0.75     0.721875 0.721875 0.74375 ]
Mean Accuracy: 0.735625


In [148]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split

# # Split the data into features (X) and target variable (y)
# X = data_enc.drop('Default payment next month', axis=1)
# y = data_enc['Default payment next month']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define the hyperparameter grid
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 5, 10],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Create the RandomForestClassifier
# rf = RandomForestClassifier(random_state=42)

# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')

# # Fit the GridSearchCV object to the training data
# grid_search.fit(X_train, y_train)

# # Print the best hyperparameters and the corresponding accuracy
# print("Best Hyperparameters: ", grid_search.best_params_)
# print("Best Accuracy: ", grid_search.best_score_)

# # Evaluate the best model on the test set
# best_model = grid_search.best_estimator_
# test_accuracy = best_model.score(X_test, y_test)
# print("Test Accuracy: ", test_accuracy)


Best Hyperparameters:  {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Accuracy:  0.7953125
Test Accuracy:  0.759375
